In [2]:
import numpy as np
from keras.models import Sequential
from keras.layers import SimpleRNN, Dense, Embedding
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

In [5]:
#Generating some example sequential data
sentences = ['I love learning','I love python','I hate school','Recurrent Nueral Networks are powerful']

In [7]:
#Tokenizing the words
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentences)
total_words = len(tokenizer.word_index) + 1
print(total_words)

12


In [8]:
# Creating input sequences and their corresponding next words
input_sequences = []
for sentence in sentences:
    tokenized_sentence = tokenizer.texts_to_sequences([sentence])[0]
    for i in range(1, len(tokenized_sentence)):
        n_gram_sequence = tokenized_sentence[:i+1]
        input_sequences.append(n_gram_sequence)
input_sequences

[[1, 2],
 [1, 2, 3],
 [1, 2],
 [1, 2, 4],
 [1, 5],
 [1, 5, 6],
 [7, 8],
 [7, 8, 9],
 [7, 8, 9, 10],
 [7, 8, 9, 10, 11]]

In [10]:
# Padding sequences for consistent input size
max_sequence_length = max([len(seq) for seq in input_sequences])
input_sequences = pad_sequences(input_sequences, maxlen=max_sequence_length, padding='pre')

In [11]:
input_sequences

array([[ 0,  0,  0,  1,  2],
       [ 0,  0,  1,  2,  3],
       [ 0,  0,  0,  1,  2],
       [ 0,  0,  1,  2,  4],
       [ 0,  0,  0,  1,  5],
       [ 0,  0,  1,  5,  6],
       [ 0,  0,  0,  7,  8],
       [ 0,  0,  7,  8,  9],
       [ 0,  7,  8,  9, 10],
       [ 7,  8,  9, 10, 11]], dtype=int32)

In [12]:
# Creating input and output data
X, y = input_sequences[:, :-1], input_sequences[:, -1]
y = to_categorical(y, num_classes=total_words)

In [14]:
# Building a simple RNN model
model = Sequential()
model.add(Embedding(input_dim=total_words, output_dim=50, input_length=max_sequence_length-1))
model.add(SimpleRNN(100, return_sequences=True))
model.add(SimpleRNN(100))
model.add(Dense(total_words, activation='softmax'))

In [15]:
#compiling the model
model.compile(optimizer = 'adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [18]:
#Training the model
model.fit(X,y,epochs=50,verbose=2)

Epoch 1/50
1/1 - 2s - loss: 2.4867 - accuracy: 0.1000 - 2s/epoch - 2s/step
Epoch 2/50
1/1 - 0s - loss: 2.3919 - accuracy: 0.5000 - 11ms/epoch - 11ms/step
Epoch 3/50
1/1 - 0s - loss: 2.3015 - accuracy: 0.7000 - 10ms/epoch - 10ms/step
Epoch 4/50
1/1 - 0s - loss: 2.2127 - accuracy: 0.6000 - 10ms/epoch - 10ms/step
Epoch 5/50
1/1 - 0s - loss: 2.1236 - accuracy: 0.4000 - 11ms/epoch - 11ms/step
Epoch 6/50
1/1 - 0s - loss: 2.0341 - accuracy: 0.4000 - 9ms/epoch - 9ms/step
Epoch 7/50
1/1 - 0s - loss: 1.9450 - accuracy: 0.4000 - 11ms/epoch - 11ms/step
Epoch 8/50
1/1 - 0s - loss: 1.8575 - accuracy: 0.4000 - 10ms/epoch - 10ms/step
Epoch 9/50
1/1 - 0s - loss: 1.7719 - accuracy: 0.4000 - 10ms/epoch - 10ms/step
Epoch 10/50
1/1 - 0s - loss: 1.6872 - accuracy: 0.4000 - 9ms/epoch - 9ms/step
Epoch 11/50
1/1 - 0s - loss: 1.6031 - accuracy: 0.4000 - 9ms/epoch - 9ms/step
Epoch 12/50
1/1 - 0s - loss: 1.5222 - accuracy: 0.5000 - 15ms/epoch - 15ms/step
Epoch 13/50
1/1 - 0s - loss: 1.4484 - accuracy: 0.6000 - 10

In [24]:
# Generating text using the trained model
seed_text = input("Enter the starting word: ")
next_words = int(input("Enter how many words to predict: "))

for _ in range(next_words):
    tokenized_seed = tokenizer.texts_to_sequences([seed_text])[0]
    tokenized_seed = pad_sequences([tokenized_seed], maxlen=max_sequence_length-1, padding='pre')
    predicted_word_index = np.argmax(model.predict(tokenized_seed), axis=-1)
    predicted_word = tokenizer.index_word[predicted_word_index[0]]
    seed_text += " " + predicted_word

print(seed_text)

Enter the starting word: hate
Enter how many words to predict: 5
1/1 [==============================] - 0s 29ms/step
hate love python are powerful powerful
